In [32]:
import requests
from bs4 import BeautifulSoup
import json
from urllib.parse import urljoin

def marca(codi):
    # URL de la pàgina web
    url = 'http://consultas2.oepm.es/ceo/jsp/busqueda/consultaExterna.xhtml?numExp=' + codi

    # Realitzar la petició GET
    response = requests.get(url)
    response.raise_for_status()  # Llença una excepció si la petició falla

    # Analitzar el contingut HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Crear el diccionari que contenirà la informació
    data = {}

    # Obtenir el text de la marca nacional
    marca_nacional = soup.find('h3').text.split(' - ')[-1]
    data['Marca nacional'] = marca_nacional

    # Funció per netejar el text
    def clean_text(text):
        return text.strip().replace('\n', ' ').replace('\r', '')

    # Buscar i afegir la informació de les dates i tipus
    for div in soup.find_all('div', class_='div2col'):
        h4 = div.find('h4')
        if h4:
            h4_text = h4.get_text(strip=True).rstrip(':')
            p = div.find('p')
            if p:
                data[h4_text] = clean_text(p.get_text(strip=True))

    # Extraure les dades del "Solicitante / Titular"
    solicitante_h4 = soup.find('h4', string='Solicitante / Titular:')
    if solicitante_h4:
        solicitante_div = solicitante_h4.find_next_sibling('div', class_='datos-sitamar')
        solicitante_info = {}
        if solicitante_div:
            for div in solicitante_div.find_all('div', class_='divSubFila'):
                field = div.find('span', class_='campo')
                value_div = div.find('div', class_='datos-sitamar-divSubRight')
                if field and value_div:
                    field_text = field.text.rstrip(':')
                    value = clean_text(value_div.get_text())
                    if "\t\t\t\t\t\t" in value:
                        if field_text == "Nombre":
                            parts = value.split("\t\t\t\t\t\t")
                            motiu = parts[0].strip()
                            nombre = parts[1].strip()
                            nom = {'Nom': nombre, 'Motiu': motiu}
                            solicitante_info[field_text] = nom
                    else:
                        solicitante_info[field_text] = value
        data['Solicitante / Titular'] = solicitante_info

    # Extraure l'Agente/Representante
    agente_h4 = soup.find('h4', string='Agente/Representante:')
    if agente_h4:
        agente_div = agente_h4.find_next_sibling('div', class_='divFila')
        if agente_div:
            agente = agente_div.get_text(strip=True)
            data['Agente/Representante'] = clean_text(agente)

    # Extraure la Clasificación de Viena
    viena_h4 = soup.find('h4', string='Clasificación de Viena:')
    if viena_h4:
        viena_div = viena_h4.find_next_sibling('div', class_='datos')
        if viena_div:
            viena = [div.get_text(strip=True) for div in viena_div.find_all('div', class_='viena')]
            data['Clasificación de Viena'] = ', '.join(viena)

    # Extraure les Clases y productos / serveis o activitats
    classes = soup.find('div', class_='ui-datalist ui-widget')
    if classes:
        classes_items = classes.find_all('li', class_='ui-datalist-item')
        classes_dict = {}
        for item in classes_items:
            codi_classe_div = item.find('div', class_='sitamar-cod-clase')
            text_classe_div = item.find('div', class_='sitamar-texto-clase')
            if codi_classe_div and text_classe_div:
                codi_classe = codi_classe_div.get_text(strip=True)
                text_classe = text_classe_div.get_text(strip=True)
                classes_dict[codi_classe] = text_classe
        data['Clases y productos / servicios o actividades solicitados'] = classes_dict

    # Extreure concedits
    concedidos_h4 = soup.find('h4', string='Clases y productos / servicios o actividades concedidos:')
    if concedidos_h4:
        concedidos_div = concedidos_h4.find_next_sibling('div', class_='datos')
        if concedidos_div:
            concedidos = concedidos_div.get_text(strip=True)
            data['Clases y productos / servicios o actividades concedidos'] = clean_text(concedidos)

    # Extraure els Actos de tramitación
    actos = []
    actos_h4 = soup.find('h4', string='Actos de tramitación:')
    if actos_h4:
        actos_div = actos_h4.find_next_sibling('div', class_='datos')
        if actos_div:
            actos_table = actos_div.find('table')
            if actos_table:
                for row in actos_table.find_all('tr')[1:]:  # Saltar el capçalera
                    cols = row.find_all('td')
                    if len(cols) > 1:
                        fecha = cols[0].get_text(strip=True)
                        acto = cols[1].get_text(strip=True)
                        actos.append({'Fecha': fecha, 'Acto': acto})
        data['Actos de tramitación'] = actos

    # Extraure les Anotaciones de pagos
    pagos = []
    pagos_h4 = soup.find('h4', string='Anotaciones de pagos:')
    if pagos_h4:
        pagos_div = pagos_h4.find_next_sibling('div', class_='datos')
        if pagos_div:
            pagos_table = pagos_div.find('table')
            if pagos_table:
                for row in pagos_table.find_all('tr')[1:]:  # Saltar el capçalera
                    cols = row.find_all('td')
                    if len(cols) > 1:
                        fecha = cols[0].get_text(strip=True)
                        pago = cols[1].get_text(strip=True)
                        pagos.append({'Fecha': fecha, 'Pago': pago})
        data['Anotaciones de pagos'] = pagos

    # Extraure la URL completa de la imatge
    imagen_div = soup.find('div', id='marco-imagen-sitamar')
    if imagen_div:
        img_tag = imagen_div.find('img')
        if img_tag:
            img_src = img_tag.get('src', '').strip()
            # Construir la URL completa de la imatge
            base_url = 'http://consultas2.oepm.es/ceo/'
            full_img_url = urljoin(base_url, img_src)
            data['Imatge'] = full_img_url

    # Crear el diccionari amb el codi com a clau principal
    result = {codi: data}

    # Convertir el diccionari a JSON
    json_output = json.dumps(result, indent=4, ensure_ascii=False)

    # Mostrar el JSON
    return json_output


In [33]:
marques = ["M2947539", "M2793400"]

# Crear el diccionari resultat
resultat = {}

for m in marques:
    # Obtenir la cadena JSON associada a la marca
    json_str = marca(m)  # `marca(m)` hauria de retornar una cadena JSON

    # Convertir la cadena JSON a un diccionari
    info = json.loads(json_str)

    # Obtenir la primera clau del diccionari `info`
    first_key = next(iter(info))

    # Afegir al diccionari resultat utilitzant la clau del diccionari `info`
    resultat[first_key] = info[first_key]

resultat

{'M2947539': {'Marca nacional': 'HOSPITAL DE LA SANTA CREU I SANT PAU',
  'Fecha Presentación': '03/05/1990 A LAS 10:31 EN Madrid',
  'Fecha presentación solicitud otorgada': '03/05/1990',
  'Tipo': 'Mixta',
  'Estado': 'En vigor',
  'Solicitante / Titular': {'Nombre': {'Nom': 'FUNDACIO PRIVADA HOSPITAL DE LA SANTA CREU I SANT PAU',
    'Motiu': 'POR TRANSFERENCIA NUM. 2000.01329 DE F. RESOL. 24/08/2000'},
   'Cuota indivisa': '0',
   'Dirección': 'AV. SANT ANTONI MARIA CLARET, 167',
   'Localidad': 'BARCELONA',
   'Provincia': 'Barcelona',
   'Código Postal': '08025'},
  'Clasificación de Viena': '24.01.25, 26.04.03',
  'Clases y productos / servicios o actividades solicitados': {'16': 'IMPRESOS, FOLLETOS, CATALOGOS, PEGATINAS Y PUBLICACIONES.',
   '42': 'SERVICIOS PROPIOS DE HOSPITALES Y CLINICAS.'},
  'Clases y productos / servicios o actividades concedidos': 'Los solicitados.',
  'Actos de tramitación': [{'Fecha': '27/09/2010',
    'Acto': 'EXPEDIENTES FUSIONADOS:M1565861   \u2003 

In [17]:
import json
import pandas as pd

# Suposant que 'json_output' és la cadena JSON que ja tens
# Convertir el JSON a un diccionari
data = json.loads(json_output)

# Crear una llista amb els primers nivells de camps de cada marca
data_list = []

for marca, details in data.items():
    # Extreure dinàmicament els camps del primer nivell
    first_level = {key: details.get(key) for key in details.keys()}
    data_list.append(first_level)

# Convertir la llista a un DataFrame de pandas
df = pd.DataFrame(data_list)

df


,Marca nacional,Fecha Presentación,Fecha presentación solicitud otorgada,Tipo,Estado,Solicitante / Titular,Clasificación de Viena,Clases y productos / servicios o actividades solicitados,Clases y productos / servicios o actividades concedidos,Actos de tramitación,Imatge
0,HOSPITAL DE LA SANTA CREU I SANT PAU,03/05/1990 A LAS 10:31 EN Madrid,03/05/1990,Mixta,En vigor,{'Nombre': {'Nom': 'FUNDACIO PRIVADA HOSPITAL ...,"24.01.25, 26.04.03","{'16': 'IMPRESOS, FOLLETOS, CATALOGOS, PEGATIN...",Los solicitados.,"[{'Fecha': '27/09/2010', 'Acto': 'EXPEDIENTES ...",http://consultas2.oepm.es/ceo/ImagenMarcaServl...
1,HOSPITAL DE LA SANTA CREU I SANT PAU,08/10/2007 A LAS 08:53 EN INTERNET,08/10/2007,Mixta,En vigor,{'Nombre': 'FUNDACIÓ PRIVADA HOSPITAL DE LA SA...,"26.04.03, 27.05.11",{'36': 'ADMINISTRACION Y ARRENDAMIENTO DE BIEN...,Los solicitados.,"[{'Fecha': '16/11/2007', 'Acto': 'PUBLICACION ...",http://consultas2.oepm.es/ceo/ImagenMarcaServl...
